# 1 PA3 - Ranking functions (45% of total PA3 grade)

In the first part of PA3, you will devise ranking functions to rank results given some queries and corresponding search results. For each query-document pair, you are provided with several features that will help you rank the documents. You are also provided with a training set consisting of query-document pairs along with their relevance values. We will be implementing **three** different ranking functions and will use the **NDCG** metric for evaluating the effectiveness of the ranking function. The estimation of parameters for the ranking functions will be done manually (i.e., no machine learning). 

More specifically, it involves the following tasks:


1. [Cosine Similarity (5%)](#V-Task1:-Cosine-Similarity-(5%)) To implement a variant of cosine similarity (with the L1-Norm) as the ranking function

2. [BM25F (15%)](#VI-Task2:-BM25F-(15%)) To implement the BM25F ranking algorithm.

3. [Smallest Window (10%)](#VII-Task3:-Smallest-Window-(10%)) Incorporate window sizes into the ranking algorithm from Task 1 (or Task 2 if you prefer). 

4. [Report (15%)](#Report-(15%)) describing your program and answer a set of questions.


__Grading for Tasks 1, 2 and 3__
- Half of your grade will be based on your model's performance on an autograder test set. Your scores will be visible to you when you submit on Gradescope, but the test set will not. 
- The other half of your grade will be based on your model's performance on a hidden test set. Your scores will only be visible to you when grades for this assignment are released
- You will get full credit for solutions that receive NDCG scores within reasonable range of the NDCG scores received by the teaching staff.

In the next part of PA3 (Learning to rank), you will explore different approaches to learn the parameters for ranking functions using machine learning. 


## Submission instructions

1\. The assignment is due before class at 4:00 pm on the due date (30th May 2019)

2\. The notebook will automatically generate **python files** in submission folder. You'll have to upload them to the PA3-code assignment on gradescope. Note that you need to upload all the individual files in the submission folder without zipping it.    

3\. While solving the assignment, do **NOT** change class and method names, autograder tests will fail otherwise. 

4\. You'll also have to upload a **PDF version** of the notebook (which would be primarily used to grade your report section of the notebook) to PA3-PDF assignment on gradescope. Note that directly converting the PDF truncates code cells. To get a usable PDF version, first click on File > Print Preview, which will open in a new tab, then print to PDF using your browser's print functionality. 

5\. Since there are two notebooks, we have included a script to help you merge them together before upload. Run
```
python pdfcat pa3-ranking.pdf pa3-learning-to-rank.pdf > pa3-solution.pdf
``` 
to generate a single concatenated pdf file and upload `pa3-solution.pdf` to gradescope.

6\. After uploading the PDF make sure you **tag all the relevant pages to each question**. We will penalize for mistagged submissions. 

7\. If you are solving the assignment in a team of two, add the other student as a group member after submitting the assignment. Do **NOT** submit the same assignment twice. 

## Setup

In [ ]:
#Load the tee magic which saves a copy of the cell when executed
%reload_ext autograding_magics
%load_ext autoreload
%autoreload 2

The `submission` folder will contain all the files to be submitted, and `base_classes` contains other class definitions which you will not submit.

In [ ]:
import os
try: 
    os.mkdir('submission')
except FileExistsError:
    pass
try:
   open('submission/__init__.py', 'x')
except FileExistsError:
   pass
try: 
    os.mkdir('base_classes')
except FileExistsError:
    pass
try:
   open('base_classes/__init__.py', 'x')
except FileExistsError:
   pass
try: 
    os.mkdir('output')
except FileExistsError:
    pass

In [ ]:
%%tee submission/imports.py

# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
from collections import OrderedDict, Counter
import math

import sys
from base_classes.load_train_data import load_train_data
from base_classes.id_map import IdMap
from base_classes.ndcg import NDCG
from base_classes.query import Query
from base_classes.document import Document
import numpy as np

# II. Data 

The data for this assignment is available as a .zip file at: http://web.stanford.edu/class/cs276/pa/pa3-data.zip. The following code puts the data folder under the current directory. We have partitioned the data into two sets for you: 
1. Training set of 731 queries (pa3.(signal|rel).train)
2. Development set of 124 queries (pa3.(signal|rel).dev)

The idea is that while tuning and maximizing performance on the training set, you should also verify how well the tuned parameters are doing on the development set to ensure you are not overfitting your model. There is a hidden test set of 124 queries which we have reserved to grade your final model. For each set, there are two types of files:

In [ ]:
import urllib.request
import zipfile

# Download dataset
data_dir = 'pa3-data'
data_url = 'http://web.stanford.edu/class/cs276/pa/{}.zip'.format(data_dir)
urllib.request.urlretrieve(data_url, '{}.zip'.format(data_dir))

# Unzip dataset
with zipfile.ZipFile('{}.zip'.format(data_dir), 'r') as zip_fh:
    zip_fh.extractall()
print('Data downloaded and unzipped to {}...\n'.format(data_dir))

# Print the directory structure
print('Directory Structure:')
print(data_dir + os.path.sep)
for sub_dir in os.listdir(data_dir):
    if not sub_dir.startswith('.'):
        print('  - ' + sub_dir)

### Signal File 
**– pa3.signal.(train|dev):** lists queries along with documents returned by a widely used search engine for each individual query (the list of documents is shuffled and is not in the same order as returned by the search engine). Each query has 10 or less documents. For example, the format for a pair of query/document (qd) is as follows.

In [ ]:
filename = os.path.join(data_dir, "pa3.signal.train")
with open(filename, 'r', encoding = 'utf8') as f:
    print(f.read()[0:1000])
print("...")

This pattern repeats for the next url until all of the urls for this query are done and then the overall pattern repeats for the next query. There is only one <b>title</b>, <b>pagerank</b>, and <b>body length</b> for each url but there can be multiple <b>header</b>, <b>body hits</b> and <b>anchor text</b> (and corresponding stanford anchor count) lines.

* The <b>body hits</b> line specifies the term followed by the positional postings list of that term in the document (sorted in increasing order).
* The <b>body length</b> line states how many terms are present in the body of the document.
* The <b>stanford anchor count</b>, specified immediately after the anchor text line, states how many anchors there are on the stanford.edu domain with that anchor text. For example, if the anchor text is “stanford math department” and the count is 9, that means there are nine links to the current page (from other pages) where the anchor text is “stanford math department”.
* The <b>pagerank</b> is an integer from 0 to 9 that signifies a query-independent quality of the page (the higher the PageRank, the better the quality of the page).

### Relevance File
**– pa3.rel.(train|dev)**: lists the relevance judgments for each of the query-document pairs in the corresponding signal file. The collected relevance data was an integer ranging from −1 to 3 with a higher value indicating that the document is more relevant to that query. We have averaged relevance scores for each query-url pair with −1 ignored. For example, the format of this document is as follows:

In [ ]:
filename = os.path.join(data_dir, "pa3.rel.train")
with open(filename, 'r', encoding = 'utf8') as f:
    print(f.read()[0:199])
print("...")

This pattern repeats for the next query until all of the queries in the file are done. The url line can be broken into the document url and the relevance judgment for the query-document pair.

The ranking functions also require certain collection-wide statistics (such as inverse document frequency) and we cannot infer this information just from the training set itself. We provide **docs.dict, terms.dict and BSBI.dict** what you generated from PA1, and leave you to calculate idf below.

# III. Normalized Discounted Cumulative Gain (NDCG)

The evaluation metric used is Normalized Discounted Cumulative Gain (NDCG) since we are using a non-binary relevance metric. Since each query has at most 10 results returned, we use NDCG for the first 10 search results.
Then, for a particular query q,
$$NDCG(q) = \frac{1}{Z} \sum_{m=1}^{p}\frac{2^{R(q,m)}-1}{log_{2}(1+m)}$$
Here, $R(q, m)$ is the relevance judgment given to document $m$ for query $q$. $Z$ is a normalization factor. It is the ideal NDCG value. The ideal NDCG (iNDCG) value is calculated by ordering the documents in decreasing order of relevance and calculating the NDCG value with $Z=1$. If iNDCG is zero, $NDCG(q) = 1$. Finally, $p$ is the number of documents that are possible matches for that query.

We can compute the NDCG for a set of queries $Q = \{q_1,...,q_m\}$ by taking the average of the NDCGs for each of the individual queries. 

The starter code [Section NDCG](#VII.1-NDCG-implementation) contains a Python implementation of NDCG which you can use directly to evaluate your ranking function on the training data. We will be using the same method to evaluate your ranking on grading data.

# IV. Ranking

## IV.1 Term Score
In the signal files of the training data, each query-document pair provides term information from five different fields: <b>url</b>, <b>title</b>, <b>headers</b>, <b>body</b> and <b>anchors</b>. Additionally each pair provides <b>pagerank</b> but we won’t be using it in cosine similarity. Even for BM25F, we will consider it separately as explained in [BM25F](#VI-Task2:-BM25F-(15%)). Each of the required ranking functions will construct a term score ($tf$) vector for each query-document pair from hits in these different fields. All of our ranking functions only care about terms that occur in the query.

The raw term score vector, $rs$, counts how many times a query term occurs in a field. For the <b>anchor</b> field, we assume that there is one big document that contains all of the anchors with the anchor text multiplied by the anchor count. A similar approach can be followed for the <b>header</b> field as well. Thus, in the $qd$ example whose term-vector is $[\text{stanford aoerc pool hours}]^T$ (which is shown in above printing of signal file), the $rs$
vector for the body field will be $[{10 \ 7 \ 1 \ 0}]^T$ as there are 10 hits for the term “stanford” in the <b>body</b> field and 7 hits for the term “aoerc” as well as 1 hits for the term “pool”. Similarly, the $rs$ vector for <b>anchor</b> field will be $[\text{0 0 0 0}]^T$ as there is no anchor for this document. For <b>anothe</b>r example 
```python
  url: https://cardinalrec.stanford.edu/facilities/aoerc/
    ...
    anchor_text: gyms aoerc
      stanford_anchor_count: 3
    anchor_text: aoerc
      stanford_anchor_count: 13
    anchor_text: http cardinalrec stanford edu facilities aoerc
      stanford_anchor_count: 4
    anchor_text: arrillaga outdoor education and recreation center aoerc link is external
      stanford_anchor_count: 1
    anchor_text: the arrillaga outdoor education and research center aoerc
      stanford_anchor_count: 2
    anchor_text: aoerc will shutdown for maintenance
      stanford_anchor_count: 2
```

The <b>anchor</b> will be $[\text{4 25 0 0}]^T$ as there is 4 stanford_anchor_count for term “stanford” and 25 stanford_anchor_count for term “aoerc”.

Finally, the $rs$ vector <br>
for the <b>title</b> field is $[\text{1 0 0 0}]^T$,<br>
for the <b>url</b> field is$[\text{1 0 0 0}]^T$, <br>
for the <b>header</b> field is $[\text{5 0 0 1}]^T$ . 

Note that in order to extract <b>url</b> hits, you will have to tokenize the url on non-alphanumeric characters. We've provided the parser code for you.

While calculating the raw term scores, we convert everything to lowercase and then calculate the counts. The <b>body_hits</b> field given in the data do not perform any stemming. However, for the other fields, you are free to experiment with different techniques like stemming etc. You may find [nltk](https://www.nltk.org/) could be useful 

## IV.2 Output Requirements
In all three tasks, the goal is to derive specific types of ranking functions based on the training data and relevance values. Once the ranking function $rf$ has been crafted, we will then pass in the test data set and your application must use $rf$ to rank the query-document pairs and output the list of documents for each query in decreasing rank order. The NDCG evaluation metric will then be applied on these lists against the evaluation provided by you in the search ratings task earlier in the course. The higher the value, the better your ranking algorithm works.

We predefine Query and Document class for you. You can load training data and construct a query dictionary by load_train_data

In [ ]:
file_name = os.path.join(data_dir, "pa3.signal.train")
query_dict = load_train_data(file_name)

```python
# Mapping of Query-url-Document. Query -> (url -> Document)
query_dict[Query("stanford aoerc pool hours")]  # Access a query 
query_dict[Query("stanford aoerc pool hours")]['an url']  # Access a document 
query_dict[Query("stanford aoerc pool hours")]['an url'].body_hits  # Access a field of document 
```

In [ ]:
sample_doc = query_dict[Query("stanford aoerc pool hours")]['http://events.stanford.edu/2014/February/18/']
print("document:", sample_doc)
print("url", sample_doc.url)
print("headers:", sample_doc.headers)
print("body_hits:",sample_doc.body_hits)

## IV.4 Build Idf Dictionary

In this section, you will need to build an idf dictionary contain idf of a term, which will be used later.

In [ ]:
%%tee submission/build_idf.py
import pickle as pkl
import math
class Idf:
    """Build idf dictionary and return idf of a term, whether in or not in built dictionary.
        Recall from PA1 that postings_dict maps termID to a 3 tuple of 
        (start_position_in_index_file, number_of_postings_in_list, length_in_bytes_of_postings_list)
        
        Remember that it's possible for a term to not appear in the collection corpus.
        Thus to guard against such a case, we will apply Laplace add-one smoothing.
        
        Note: We expect you to store the idf as {term: idf} and handle term which is not in posting_list

        Hint: For term not in built dictionary, we should return math.log10(total_doc_num / 1.0).
    """
    def __init__(self):
        """Build an idf dictionary"""
        try:
            # We provide docs.dict, terms.dict and BSBI.dict what you generated from PA1
            with open("pa3-data/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            print("Total Number of Docs is", self.total_doc_num)

            with open("pa3-data/terms.dict", 'rb') as f:
                terms = pkl.load(f)
            self.total_term_num = len(terms)
            print("Total Number of Terms is", self.total_term_num)

            with open('pa3-data/BSBI.dict', 'rb') as f:
                postings_dict, termsID = pkl.load(f)

            self.idf = {}
            ### Begin your code

            ### End your code
        except FileNotFoundError:
            print("doc_dict_file / term_dict_file Not Found!")

    def get_idf(self, term = None):
        """Return idf of return idf of a term, whether in or not in built dictionary.
        Args:
            term(str) : term to return its idf
        Return(float): 
            idf of the term
        """
        ### Begin your code

        ### End your code

In [ ]:
my_idf = Idf()
my_idf.get_idf("data")
assert len(my_idf.idf) == 347071, 'Not matching with expected length of idf.' 
assert my_idf.get_idf("bilibalabulu") > 4.9, "Not handle unseen term or give wrong value"
assert my_idf.get_idf("data") < my_idf.get_idf("radiology"), 'idf of rarer terms should be larger than common terms.'
assert my_idf.get_idf("to") < my_idf.get_idf("design"), 'idf of rarer terms should be larger than common terms.'

# V Task1: Cosine Similarity (5%)

The first task is to implement a variant of cosine similarity (with the L1-Norm) as the ranking function. This essentially involves constructing the <b><i>document vector</i></b> and the <b><i>query vector</i></b> and then taking their dot product. Recall from Figure 6.15 in the textbook that in order to construct the vectors, we need to decide on how we compute a term frequency, a document frequency weighting, and a normalization strategy. Let’s discuss these for both the vectors separately.
<img src="fig/IIR_fig_6.15.png">
Figure is from Pg.128 http://nlp.stanford.edu/IR-book/pdf/06vect.pdf

Note: We will only grade Task 1 on default parameter to check the correctness of your implementation. But it could be helpful to do parameter tuning on it and have a sense of the importance of each field. You will need that in Task 2.

## V.1 Query vector 

* Term frequency<br>
The raw term frequencies can be computed using the query (should be 1 for most queries but not necessarily true). Again, you can use either the raw frequencies or sublinearly scale them.


* Document frequency<br>
Each of the terms in <i>qv</i> should be weighted using the idf value for each of the terms in the query. Computing the idf above from the corpus from PA1 to determine how many documents contain the query terms. One issue is that it is possible for a query term <i>t</i> to not appear in the collection corpus and it is not possible to evaluate ${\text{idf}_t}$. In such a case, we will apply the Laplace add-one smoothing technique learned earlier in the course 3. (This essentially assumes the existence of a hypothetical dummy document that contains all possible terms, and therefore, adds 1 to each numerator and denominator with the idft formula.)


* Normalization<br>
No normalization is needed for query length because any query length normalization applies to all docs and so is not relevant to ranking.

**Note**: We ask you to implement the b-t-n (boolean-idf-none) scheme for query vector and check the correctness of your scorer based on this default setting. You could select other reasonable scheme to increase the performance.


## V.2 Document vector
* Term frequency<br>
We compute the raw term frequencies for each query term in the different fields using the method described in [Section IV.1](#IV.1-Term-Score) . For each of the fields, we can compute the <i>tf</i> vector, either using the raw scores themselves or by applying sublinear scaling on the raw scores. In sublinear scaling, we have $tf_i = 1 + log(rs_i)$ if $rs_i > 0$ and $0$ otherwise. Thus, the <i>tf</i> vector for the <b>body</b> field for qd will be $[\text{1+log(10)  1+log(7)  1+log(1)  0}]^T$ . 
More information about sublinear tf scaling is described in <a href="http://nlp.stanford.edu/IR-book/pdf/06vect.pdf"> Page 126 Section 6.4.1 of the textbook</a>.


* Document frequency<br>
We will not use any document frequency in the document vector. Instead, it is incorporated in the query vector as described below.


* Normalization<br>
We cannot use cosine normalization as we do not have access to the contents of the document and, thus, do not know what other terms (and counts of those terms) occur in the <b>body</b> field. As a result, we use length normalization instead. Moreover, since there can be huge discrepancies between the lengths of the different fields, we divide all fields by the same normalization factor, the body length. <br> Note that some documents have a body length of 0, so you will have to smooth them somehow. A good strategy is to add a value, say 500, to the body length of each document. You can experiment with this value or with other smoothing strategies and report them.

Note: We ask you to implement the n-n-n* (natural-no- some normalization*) scheme for document vector for and check the correctness of your scorer based on this default setting. You could select other reasonable scheme to increase the performance.

**Hint:** The normalizaton of document vector of task 1 and task 2 are different but the tasks could share same term frequency and document frequency

Note that to fully test the correctness of your scorer, we provide the defaut weight scheme for query vector and doc vector. You should **implement the defaut ones and any other variants that you believe will increase the performance**.

## V.3 Abstract Scorer and Baseline Scorer

In [ ]:
# We use a sample q and d to help/assert your score implementation 
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/'] # example that has body_hits
# d = query_dict[q]['https://cardinalrec.stanford.edu/facilities/aoerc/']  # example that has anchors
print("Query q: ", q)
print("Document d: ", d)

In [ ]:
%%tee submission/ascore.py
import math
from collections import Counter
class AScorer:
    """ An abstract class for a scorer. 
        Implement query vector and doc vector.
        Needs to be extended by each specific implementation of scorers.
    """
    def __init__(self, idf, query_weight_scheme=None, doc_weight_scheme=None): #Modified
        self.idf = idf
        self.TFTYPES = ["url","title","body_hits","header","anchor"]
        
        self.default_query_weight_scheme = {"tf": 'b', "df": 't', "norm": None} # boolean, idf, none
        self.default_doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}   # natural, none
        
        self.query_weight_scheme = query_weight_scheme if query_weight_scheme is not None \
                                   else self.default_query_weight_scheme #Modified (added)
        self.doc_weight_scheme = doc_weight_scheme if doc_weight_scheme is not None \
                                 else self.default_doc_weight_scheme #Modified (added)

    def get_sim_score(self, q, d):
        """ Score each document for each query.
        Args:
            q (Query): the Query
            d (Document) :the Document

        Returns:
            pass now, will be implement in task 1, 2 and 3
        """        
        raise NotImplementedError

    # Include any initialization and/or parsing methods that 
    # you may want to perform on the Document fields prior to accumulating counts.
    # See the Document class to see how the various fields are represented
    # We have provided a few parser functions for you. Feel free to change them, and add more if you find its useful

    ### Begin your code

    ### End your code
    def parse_url(self, url, token=False):
        """Parse document's url. Return Counter of url's tokens"""
        # token indicate whether we want the raw token or Counter of it
        if url:
            url_token_in_term = url.replace("http:",".").replace('/','.').replace('?','.') \
                                   .replace('=','.').replace("%20",".").replace("...",".").replace("..",".")\
                                   .lower();
            url_token = url_token_in_term.split('.')[1:]
            if token:
                return url_token 
            else:
                return Counter(url_token)
        return Counter([])

    def parse_title(self, title, token=False):
        """Parse document's title. Return Counter of title's tokens"""
        if title:
            if token:
                return title.split(" ") 
            else:
                return Counter(title.split(" "))
        else:
            return Counter([])

    def parse_headers(self, headers):
        """Parse document's headers. Return Counter of headers' tokens"""
        headers_token = []
        if headers is not None:
            for header in headers:
                header_token = header.split(" ")
                headers_token.extend(header_token)
        return Counter(headers_token)

    def parse_anchors(self, anchors):
        """Parse document's anchors. Return Counter of anchors' tokens"""
        anchor_count_map = Counter({})
        if anchors is not None:
            for anchor in anchors:
                count = anchors[anchor]
                anchor_tokens = anchor.split(" ")
                for anchor_token in anchor_tokens:
                    if(anchor_token in anchor_count_map.keys()):
                        anchor_count_map[anchor_token] += count
                    else:
                        anchor_count_map[anchor_token] = count           
        return anchor_count_map
 
    def parse_body_hits(self, body_hits):
        """Parse document's body_hits. Return Counter of body_hits' tokens"""
        body_hits_count_map = Counter({})
        if body_hits is not None:
            for body_hit in body_hits:
                body_hits_count_map[body_hit] = len(body_hits[body_hit])
        return body_hits_count_map
    
    
    def get_query_vector(self, q, query_weight_scheme=None):

        """ Handle the query vector. 
        1. get term freq 2. get doc freq 3. normalization
        Refer to above SMART notificaton and figure
        
        Compute the raw term (and/or sublinearly scaled) frequencies
        Additionally weight each of the terms using the idf value of the term in the query 
        (we use the PA1 corpus to determine how many documents contain the query terms 
        which is calculated above and stored in self.idf).
        
        Note that no normalization is needed for query length 
        because any query length normalization applies to all docs and so is not relevant to ranking.
        
        Args:
            q (Query): Query("some query")
            
        Returns:
            query_vec (dict):  the query vector
        """  
       
        if query_weight_scheme is None:
            query_weight_scheme = self.query_weight_scheme #modified
            
        query_vec = {}
        ### Begin your code

        ### End your code
        return query_vec
    
    def get_doc_vector(self, q, d, doc_weight_scheme=None):
        
        """get term freqs for documents
        You will need to 
        1. Initialize tfs for tf types (as in self.TFTYPES)
        2. Initialize tfs for query_words
        3. Tokenize url, title, and headers, anchors, body_hits if exits
        4. (we've already provided parse functions above)
        5. Loop through query terms increasing relevant tfs
        
        Args:
        q (Query) : Query("some query")
        d (Document) : Query("some query")["an url"]
        
        Returns:
        doc_vec (dict) :A dictionary of doc term frequency:
                    tf type -> query_word -> score
                    For example: the output of document d
                    Should be look like "{'url': {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                                     'title': {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},...""
        """
        if doc_weight_scheme is None:
            doc_weight_scheme = self.doc_weight_scheme #modified
            
        doc_vec = {} 
        
        ### Begin your code

        ### End your code
        
        # Normalization
        if doc_weight_scheme['norm']:
            norm_func = doc_weight_scheme["norm"]
            doc_vec = norm_func(q, d, doc_vec)
        return doc_vec
        
        
    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalize the doc vector
        Task 1 and 2 will use different normlization. You can also try other different normalization methods.
        Args: 
            doc_vec (dict) : the doc vector
            q (Query) : the query
            d (Document) : the document
        """
        raise NotImplementedError
        
    # For the learning-to-rank ipython notebook, you may choose to define additional function(s)
    # below for various possible kinds of normalization. 
    # You will not need to fill this section out for the "ranking" notebook. 

    ### Begin your code

    ### End your code 
    
    def get_net_score(self, q, query_vec, d, doc_vec):
        """ calculate net score
        Args:
            q (Query) : the query
            query_vec (dict) : the query vector
            d (Document) : the document
            doc_vec (dict) : the document vector
        Return:
            score (float) : the net score
        """
        raise NotImplementedError

Free free to compare your get_doc_vector result with above instruction and your own understanding. 
We did not use  other techniques such as stemming. But free to do that yourself to increase the performance

In [ ]:
a_scorer = AScorer(my_idf)
query_vec = a_scorer.get_query_vector(q, None) 
print(query_vec)
doc_vec = a_scorer.get_doc_vector(q, d, None) 
doc_vec

## IV.4 Baseline Score

Here we provide the a baseline score to partially test your implementation of get_query_vector and get_doc_vector.

In [ ]:
%%tee base_classes/baseline_score.py
class BaselineScorer(AScorer):
    def __init__(self, idf):
        super().__init__(idf)
    
    def get_sim_score(self, q, d):
        q_vec = self.get_query_vector(q)
        d_vec = self.get_doc_vector(q, d)
        score = 0
        if 'body_hits' in d_vec.keys():
            for term in d_vec['body_hits'].keys():
                score += d_vec['body_hits'][term]
        return score

In [ ]:
baseline_scorer = BaselineScorer(my_idf)
print('query vector: ',  baseline_scorer.get_query_vector(q))
print('doc vector', baseline_scorer.get_doc_vector(q, d))
assert baseline_scorer.get_sim_score(q, d) == 18, "Similarity scorer using default weight scheme for q and d \
                                                   does not match with our results"

For a document $d$ and query $q$, if $qv_q$ is the query vector and $tf_{d,u}$, $tf_{d,t}$, $tf_{d,b}$, $tf_{d,h}$ and $tf_{d,a}$ are the term score vector for the <b>url</b>, <b>title</b>, <b>body</b>, <b>header</b> and <b>anchor fields</b>, respectively, then the net score is
$$qv_q \cdot (c_u \cdot tf_{d,u} + c_t \cdot tf_{d,t} + c_b \cdot tf_{d,b} + c_h \cdot tf_{d,h} + c_a \cdot tf_{d,a})$$

Here, $c_u$, $c_t$, $c_b$, $c_h$ and $c_a$ are the weights given to <b>url</b>, <b>title</b>, <b>body</b>, <b>header</b> and <b>anchor fields</b>, respectively.

The goal is to determine the weights for all 5 fields (and, thus, the ranking function using cosine similarity) so that the NDCG function is of an optimal value when run on the test set. You will use the training set given to derive the above parameters.

**Hint**: Note that the absolute values of weights won’t matter as they will be the same for all documents, only the relative weights for different fields is important; i.e. you can multiply each weight by a constant and the ranking will remain the same. In order to estimate the relative weights, try to reason the relative importance of the different fields.

In [ ]:
default_params_cosine = {
    "url_weight" : 10,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "smoothing_body_length" : 800,
}

In [ ]:
%%tee submission/cosine_score.py
from submission.ascore import * #modified
class CosineSimilarityScorer(AScorer):

    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None): #Modified
        # query_dict is unnecessary for CosineSimilarityScorer,
        # but it's useful for child class SmallestWindowScorer
        super().__init__(idf, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme) #Modified
        self.url_weight = params["url_weight"]
        self.title_weight  = params["title_weight"]
        self.body_hits_weight = params["body_hits_weight"]
        self.header_weight = params["header_weight"]
        self.anchor_weight = params["anchor_weight"]
        self.smoothing_body_length = params["smoothing_body_length"]
        
    def get_net_score(self, q, query_vec, d, doc_vec):
        """ calculate net score
        Args:
            q (Query) : the query
            query_vec (dict) : the query vector
            d (Document) : the document
            doc_vec (dict) : the document vector
        Return:
            score (float) : the net score
        """
        ### Begin your code

        ### End your code
        return score
    
    
    ## Normalization
    def L1_normalize_doc_vec(self, q, d, doc_vec): 
        """ Normalize the doc vector
        Note that we should give uniform normalization to all fields
        as discussed in Session V.2 Document vector - Normalization.
        Args: 
            q (Query) : the query
            d (Document) : the document
            doc_vec (dict) : the doc vector
        Return:
            doc_vec (dict) : the doc vector after normalization
        """
        ### Begin your code

        ### End your code    
        
        
    def get_sim_score(self, q, d):
        """ Get the similarity score between a document and a query.
        Args:
            q (Query) : the query
            d (Document) : the document
            
        Return: the similarity score of q and d
        """
        query_vec = self.get_query_vector(q) 
        # Define normalizattion functon here or directly pass in normalize_func as shown in below cell
        self.doc_weight_scheme['norm'] = self.L1_normalize_doc_vec #modified
        # Normalization
        norm_doc_vec = self.get_doc_vector(q, d, self.doc_weight_scheme) #modified
        return self.get_net_score(q, query_vec, d, norm_doc_vec)

In [ ]:
cs = CosineSimilarityScorer(my_idf, query_dict, default_params_cosine)

query_weight_scheme = {"tf": 'b', "df": 't', "norm": None} 
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": cs.L1_normalize_doc_vec}

print('QUERY Vector: ',  cs.get_query_vector(q, query_weight_scheme), '\n')

print('unnormalized doc vector', cs.get_doc_vector(q, d, None), '\n')
print('score after normalize doc vector', cs.get_sim_score(q, d), '\n')

# VI Task2: BM25F (15%) 

The second task is to implement the BM25F ranking algorithm. The algorithm is decribed in detail in the lecture slides. Specifically, you should have a look at BM25F related slides of [04/25 lecture](http://web.stanford.edu/class/cs276/19handouts/lecture7-probir-1per.pdf) before reading further. Here, instead of using the term scores from [Section IV.1](##IV.1-Term-Score), we use field-dependent normalized term frequency ($ftf$). Thus, for a given term $t$ and field $f \in \{url, header, body, title, anchor\}$ in document $d$, 

\begin{equation} 
ftf_{d,f,t} = \frac{tf_{d,f,t}}{1 + B_f((\text{len}_{d,f} / \text{avlen}_f) - 1)} 
\tag{1}
\end{equation}
where $tf_{d,f,t}$ is the raw term frequency of $t$ in field $f$ in document $d$, $len_{d,f}$ is the length of $f$ in $d$ and $avlen_f$ is the average field length for $f$. The variables $avlen_{body}$, $avlen_{url}$, $avlen_{title}$, $avlen_{header}$ and $avlen_{anchor}$ can be computed using the training set. $B_f$ is a field-dependent parameter and must be tuned for this task. If $avlen_f$ is zero (should not happen in this dataset), then $ftf_{d,f,t} = 0$.

Then, the overall weight for the term $t$ in document $d$ among all fields is 
\begin{equation} \label{wtd}
w_{d,t} = \sum_{f}W_f \cdot ftf_{d,f,t}
\tag{2}
\end{equation}
Here, $W_f$ is also a field-dependent parameter that determines the relative weights given to each field. This value is similar in theory to the tuning parameters for Task 1. 


Since, we also have a non-textual feature, in the form of <b>pagerank</b>, we incorporate it into our ranking function using the method described in the BM25 lecture regarding ranking with non-textual features.


Therefore, the overall score of document $d$ for query $q$ is then:
\begin{equation} \label{bmeqn}
\sum_{t \in q} \frac{w_{d,t}}{K_1 + w_{d,t}}idf_t + \lambda V_{j}(f)
\tag{3}
\end{equation}
where $K_1$ is also a free parameter and $V_{j}$ can be a log/saturation/sigmoid function as mentioned in the slides (you will need to experiment with the other parameter $\lambda^\prime$ used by the $V_{j}$ function).

Thus, for this task, there are a minimum of 13 parameters to optimize, namely $B_{url}, B_{title}, B_{header}$, $B_{body}, B_{anchor}$, $W_{url}, W_{title}, W_{header}$, $W_{body}, W_{anchor}$, $\lambda, \lambda^\prime$ and $K_1$. Additionaly, you also have to select the $V_{j}$ function appropriately.

While in theory, BM25F should give a better NDCG value as it incorporates a lot of more information, this need not necessarily be the case. 

**Hint**: The weight values obtained in Task1 may be a good starting point for this task. Again note that the weights will depend on the "importance" of the fields. Moreover, as mentioned in the slides, log(pagerank) works well in practice but you should try other functions as well and see how they work.


In [ ]:
default_params_bm25f = {
    "url_weight" : 0.1,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "b_url" : 0.1,
    "b_title" : 0.1,
    "b_header" : 0.1,
    "b_body_hits" : 0.1,
    "b_anchor" : 0.1,
    "k1": 0.1,
    "pagerank_lambda" : 0.1,
    "pagerank_lambda_prime" : 0.1, 
}

In [ ]:
%%tee submission/params_bm25f.py
### Begin your code
params_bm25f = {
    
}
### End your code

In [ ]:
%%tee submission/bm25f_score.py
from submission.ascore import * #modified
class BM25FScorer(AScorer):

    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None): #modified
        super().__init__(idf, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme) #modified
        self.query_dict = query_dict
        
        self.url_weight = params['url_weight']
        self.title_weight  = params['title_weight']
        self.body_hits_weight = params['body_hits_weight']
        self.header_weight = params['header_weight']
        self.anchor_weight = params['anchor_weight']
        # bm25 specific weights
        self.b_url = params['b_url']
        self.b_title = params['b_title']
        self.b_header = params['b_header']
        self.b_body_hits = params['b_body_hits']
        self.b_anchor = params['b_anchor']
        self.k1 = params['k1']
        self.pagerank_lambda = params['pagerank_lambda']
        self.pagerank_lambda_prime = params['pagerank_lambda_prime']

        # BM25F data structures feel free to modify these
        # Document -> field -> length
        self.length = {}
        self.avg_length = {}
        self.pagerank_scores = {}
        
        self.calc_avg_length()
        
    def calc_avg_length(self):
        """ Set up average lengths for BM25F, also handling PageRank. 
        You need to 
        Initialize any data structures needed.
        Perform any preprocessing you would like to do on the fields.
        Handle pagerank
        Accumulate lengths of fields in documents. 
        Hint: You could use query_dict
        """
        ### Begin your code

        ### End your code
        
    def get_net_score(self, q, query_vec, d, doc_vec):
        """ Compute the overall score using above equation
        Args:
            q (Query) : the query
            query_vec (dict) : the query vector
            d (Document) : the document
            doc_vec (dict) : the doc vector
        Return:
            score (float) : the net score
        """
        ### Begin your code

        ### End your code
        return score
    
    
    def bm25f_normalize_doc_vec(self, q, d, doc_vec):
        """ Normalize the raw term frequencies in fields in document d 
            using above equation (1).
        Args:
            q (Query) : the query       
            d (Document) : the document
            doc_vec (dict) : the doc vector
        Return: 
            doc_vec (dict) : the doc vector after normalization
        """
        ### Begin your code

        ### End your code    
        
    def get_sim_score(self, q, d):
        """ Get the similarity score between a document and a query.
        Args:
            d (Document) : the document
            q (Query) : the query
            
        Return:
            the similarity score
        """
        query_vec = self.get_query_vector(q)
        # Define normalizattion functon here or directly pass in normalize_func as shown in below cell
        self.doc_weight_scheme['norm'] = self.bm25f_normalize_doc_vec #modified
        norm_doc_vec = self.get_doc_vector(q, d, self.doc_weight_scheme) #modified
        # Normalization
        return self.get_net_score(q, query_vec, d, norm_doc_vec)

In [ ]:
bm25f_scorer = BM25FScorer(my_idf, query_dict, default_params_bm25f)

# You can directly pass in normalize_func here or define normalizattion functon as shown in above cell
query_weight_scheme = {"tf": 'b', "df": 't', "norm": None} 
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": bm25f_scorer.bm25f_normalize_doc_vec}


print('QUERY Vector: ',  bm25f_scorer.get_query_vector(q, query_weight_scheme), '\n')
print('unnormalized doc vector', bm25f_scorer.get_doc_vector(q, d, None), '\n')
print('score after normalize doc vector', bm25f_scorer.get_sim_score(q, d), '\n')

# VII Task3: Smallest Window (10%)

The final task is to incorporate window sizes into the ranking algorithm from Task 2 (or Task 1 if you prefer). For a given query, the smallest window $w_{q,d}$ is defined to be the smallest sequence of tokens in document $d$ such that all of the terms in the query $q$ for are present in that sequence. A window can only be specific to a particular field and for anchor fields, all of the terms in $q$ must be present within a particular anchor text (i.e, if one term occurs in one anchor text and another term in a different anchor text, then it cannot be considered for a window). If $d$ does not contain any of the query terms or a window cannot be found, then $w_{q,d} = \infty$. Intuitively, the smaller $w_{q,d}$ is, the more relevant the document should be to the query. Thus, we can also multiply the document score (from Task 1 or Task 2) by a boost based on $w$ such that:

* If $w_{q,d} = \infty$, then the boost is 1. 
* If $w_{q,d} = |Q|$ where $Q$ are the unique terms in $q$, then we multiply the score by some factor $B$. 
* For values of $w_{q,d}$ between the query length and infinite, we provide a boost between $B$ and 1. The boost should decrease rapidly with the size of $w_{q,d}$ and can decrease exponentially or as $\frac{1}{x}$. 

Thus, for this task, there are either 7 or 15 parameters to optimize, depending on whether you decide to modify cosine similarity or BM25F. The choice of function to use when the window size is not the same as the query length is another factor to also consider. 

In [ ]:
# params depends on the scorer you are using
default_params_window = {
 'B': 1.16,
 'url_weight': 0.1,
 'title_weight': 0.1,
 'body_hits_weight': 0.1,
 'header_weight': 0.1,
 'anchor_weight': 0.1,
 'b_url': 0.1,
 'b_title': 0.1,
 'b_header': 0.1,
 'b_body_hits': 0.1,
 'b_anchor': 0.1,
 'k1': 0.1,
 'pagerank_lambda': 0.1,
 'pagerank_lambda_prime': 0.1
}

In [ ]:
%%tee submission/params_window.py
# params depends on scorer you are using
### Begin your code
params_window = {
    
}
### End your code

In [ ]:
%%tee submission/smallest_window_score.py
from submission.cosine_score import * #modified
from submission.bm25f_score import * #modified
class SmallestWindowScorer(BM25FScorer): 
    """
     A skeleton for implementing the Smallest Window scorer in Task 3.
     Note: The class provided in the skeleton code extends BM25Scorer in Task 2. 
     However, you don't necessarily have to use Task 2. (You could also use Task 1, 
     in which case, you'd probably like to extend CosineSimilarityScorer instead.)
     Also, feel free to modify or add helpers inside this class.
     
     Note: If you plan to use cosine similarity scorer
             - change parent class to CosineSimilarityScorer 
             - change normalization method in get_sim_score 
    """
    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None): #modified
        super().__init__(idf, query_dict, params, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme) #modified
        self.query_dict = query_dict
        
        # smallest window specific weights
        self.B = params["B"]
    
    # Write helper functions here
    ### Begin your code

    ### End your code
        
    def get_boost_score(self, q, d):
        """ calculate boost score based on smallest window size"""
        ### Begin your code

        ### End your code
    

    def get_sim_score(self, q, d):
        """ Get the similarity score between a document and a query.
        Args:
            d (Document) : the document
            q (Query) : the query
            
        Return:
            the raw similarity score times boost
        """
        boost = self.get_boost_score(q, d)
        query_vec = self.get_query_vector(q)
        # Define normalizattion functon here or directly pass in normalize_func as shown in below cell
        # Depends on which parent class you are using 
        self.doc_weight_scheme['norm'] = self.bm25f_normalize_doc_vec #modified
        norm_doc_vec = self.get_doc_vector(q, d, self.doc_weight_scheme) #modified
        raw_score = self.get_net_score(q, query_vec, d, norm_doc_vec)
       
        return boost * raw_score

In [ ]:
smallest_window_scorer = SmallestWindowScorer(my_idf, query_dict, default_params_window)

# You can directly define weight_scheme here 
query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}  
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": smallest_window_scorer.bm25f_normalize_doc_vec}

print('QUERY Vector: ',  smallest_window_scorer.get_query_vector(q, query_weight_scheme), '\n')
print('unnormalized doc vector', smallest_window_scorer.get_doc_vector(q, d, None), '\n')
print('score after normalize doc vector', smallest_window_scorer.get_sim_score(q, d), '\n')

# VIII Rank and Evaluate

In the rank class, you need to construct the ranking results based on different scores. 

In [ ]:
%%tee submission/rank.py
from collections import Counter
from collections import OrderedDict

class Rank:
    def score(self, query_dict, score_type, idf, params):
        
        """ Call this function to score and rank documents for some queries, 
            with a specified scoring function.
        Args:
            query_dict (dict) :  Mapping of Query-url-Document.
            score_type (str) : "baseline"  "cosine" "bm25f" "window" "extra"
            idf (dict) : term-idf dictionary
            params(dict) : parames for scorer
        Return 
            query_rankings (dict) : a mapping of queries to rankings
        """
        if score_type == "baseline": scorer = BaselineScorer(idf)
        elif score_type == "cosine": scorer = CosineSimilarityScorer(idf, query_dict, params)
        elif score_type == "bm25f": scorer = BM25FScorer(idf, query_dict, params)
        elif score_type == "window": scorer = SmallestWindowScorer(idf, query_dict, params)
        elif score_type == "extra": scorer = ExtraCreditScorer(idf, query_dict, params) 
        else: print("Wrong score type!")

        # loop through urls for query, getting scores
        query_rankings = {}
        for query in query_dict.keys():
            doc_and_scores = {}
            # rank the urls based on scores
            ### Begin your code

            ### End your code
        
        return query_rankings
    
    def rank_with_score(self, input_dict):
        
        """ Call this function to accept dictionary with an ordered ranking of queries. 
        You will need to implement this function for the learning-to-rank ipython notebook. 
        Note that this function will likely replicate code from the score function above.
        Args:
            input_dict (dict) :  Mapping of Query-url-score.
        Return 
            query_rankings (dict) : An ordered dictionary of Query->url->score (ordering done for each query)
        
        """
        # loop through urls for query, getting scores
        query_rankings = {}
        for query in input_dict.keys():
            url_and_scores = {}
            # sort the urls based on scores
            ### Begin your code

            ### End your code
        return query_rankings
    
    def write_ranking_to_file(self, query_rankings, ranked_result_file):
        with open(ranked_result_file, "w") as f:
            for query in query_rankings.keys():
                f.write("query: "+ query.__str__() + "\n")
                for res in query_rankings[query]:
                
                    url_string = "  url: " + res.url + "\n" + \
                                "    title: " + res.title + "\n" +\
                                "    debug: " + "\n" 
                    
                    f.write(url_string)
                    
        print("Write ranking result to " + ranked_result_file + " sucessfully!")     
 

Write your result to file to check your implementation

In [ ]:
# This is an example of how to use ranking class 
# Ranking 
r = Rank()
query_rankings = r.score(query_dict, 'bm25f', my_idf, default_params_bm25f)
ranked_result_file = os.path.join("output", "ranked_result_bm25f")
r.write_ranking_to_file(query_rankings, ranked_result_file)

## VIII.1 NDCG implementation

We provide the NDCG implementation for you in 'base_classes/ndcg.py'. You can use them to evaluate your results and do paramater tuning.

This is an example of how to use the Rank class and NDCG class to evaluate your scorer and ranking function.

In [ ]:
# This is an example of how to use ranking class and NDCG

# Load data and generate query dict
signal_file_name = "pa3.signal.train"
query_dict = load_train_data(os.path.join(data_dir, signal_file_name))

# Ranking 
r = Rank()
query_rankings = r.score(query_dict, 'cosine', my_idf, default_params_cosine)
ranked_result_file = os.path.join("output", "ranked_result_cosine")
r.write_ranking_to_file(query_rankings, ranked_result_file)

# NDCG
ndcg = NDCG()
rel_filename = 'pa3.rel.train'
rel_file = os.path.join(data_dir, rel_filename)

ndcg.get_rel_scores(rel_file)
ndcg.read_ranking_calc(ranked_result_file)

# You can also write the ndcg result to file
ndcg_result_file = os.path.join("output", "ndcg_result_cosine") 
ndcg.write_ndcg_result(ndcg_result_file)

# calculate average NDCG
avg_ndcg = ndcg.get_avg_ndcg()
print(avg_ndcg)

Make sure to use NDCG on **both training and development** sets for all tasks to do parameter tuning. 



Your solution will be evaluated on a hidden test set, and full credit will be given to models that are within 1% of the staff implementation's test-set . Typically, the higher the better. 


# Report (15%)

**1. (1%) Report NDCG on both training and development sets for all tasks.**
> *Your answer here*

**2. (2%)  For the three tasks, you should report all final model parameter values. Describe the intuition when tuning your models, and why those weights work in getting a good score. Were there any particular properties about the documents that allowed a higher weight to be given to one field as opposed to another?**
> *Your answer here*

**3. (3%)  In BM25F, in addition to the weights given to the fields, there are 8 other parameters, $B_{url}$, $B_{title}$, $B_{header}$, $B_{body}$, $B_{anchor}$, λ, λ′ and K1. How do these parameters affect the ranking function?**
> *Your answer here*

**4. (3%)In task 1, you may either use raw frequencies or sublinearly scale them to compute term frequency. Please report your choice and the reasons behind them. For BM25F, why did you select a particular $V_j$ function?**
> *Your answer here*

**5.(3%)  Briefly describe your design of smallest window. For a function that includes the smallest window as one component, how does varying B and the boost function change the performance of the ranking algorithm?**
> *Your answer here*

**6.(3%)  What other metrics, not used in this assignment, could be used to get a better scoring function from the document? The metrics could either be static (query-independent, e.g. document length) or dynamic (query-dependent, e.g. smallest window).**
> *Your answer here*

###  You are all done in the PA3 part 1. Now, it's time to start part 2 to explore different approaches to learn the parameters for ranking functions using machine learning. 